In [1]:
#導入套件
import pandas as pd
import numpy as np
data = pd.read_csv('purchase_data.csv') #輸入原始資料
data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


In [2]:
#Player Count
#Display the total number of players

# 清除column SN 裡的重複值 。
unique_sn = data.drop_duplicates(subset = 'SN') #sn 代表 screen name 
## output 結果
player_count = pd.DataFrame([unique_sn['SN'].count()],columns = ['Total players'])
player_count

,Total players
0,576


In [3]:
#Purchasing Analysis (Total)
#Run basic calculations to obtain number of unique items, average price, etc.
#Create a summary dataframe to hold the results
#Display the summary dataframe

#取出 column Item ID，並清除重複值
unique_ID = data['Item ID'].drop_duplicates()
unique_ID = pd.DataFrame(unique_ID)
unique_ID.count()
price = data['Price'] 

#dict 裡的 key 會變成 dataframe 的 column name
purchase_analysis  = {'Number of Unique Items': unique_ID.count(),'Average Price':"$%.2f" % (price.sum() / price.count())
    ,'Number of Purchases': price.count(),'Total Revenue':"$%.2f" % price.sum()}
purchase_analysis = pd.DataFrame(purchase_analysis).rename(index={'Item ID': "0"})
purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [4]:
#Gender Demographics
#Percentage and Count of Male Players
#Percentage and Count of Female Players
#Percentage and Count of Other / Non-Disclosed

#利用df.groupby 將 unique_sn 依照 Gender 做個數加總
Gender_demo = unique_sn.groupby("Gender",as_index=False).size()
Gender_demo = pd.DataFrame(Gender_demo)

#取出 column gender 放入list，並將 column gender 移除 
index_name = Gender_demo['Gender'].values.tolist()
Gender_demo = Gender_demo.drop('Gender', axis = 1)

# 製造 dict new_index，並對 Gender_demo column 和 index 重新命名
new_index = []
i = 0
while i < len(index_name):
    a = []
    a.append(i)
    a.append(index_name[i])
    new_index.append(a)
    i = i + 1

new_index = dict(new_index)
Gender_demo = Gender_demo.rename(index = new_index, columns = {"size":"Total Count"}) 

#插入各類別數量的百分比
Gender_demo['Percentage of Players'] = (100 *  Gender_demo / Gender_demo.sum()).round(2) 

#調整 Gender demo column 和 index 的順序
column_order = Gender_demo.columns.tolist()
column_order = column_order[-1:] + column_order[:-1]
index_order = index_name[1:2] +  index_name[0:1] + index_name[2:]
Gender_demo = Gender_demo[column_order].reindex(index = index_order)
Gender_demo

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11


In [5]:
#Purchasing Analysis (Gender)
#Run basic calculations to obtain below values by gender
#Purchase Count ，Average Purchase Price ，Total Purchase Value ，Average Purchase Total per Person by Gender
#Create a summary dataframe to hold the results
#Display the summary dataframe

#對 unique_sn 執行 df.groupby ，得出 每種 Gender 的 player count
Unique_player = unique_sn.groupby("Gender").size()
Unique_player  = pd.DataFrame(Unique_player,columns = ['Player Count'])

#對 data 執行 df.groupby ， 得出 每種 Gender 的 price 總和 ，並將 column 重新命名為 Total Purchase Value
Pur_analy = data.groupby(['Gender']).Price.sum() 
Pur_analy = pd.DataFrame(Pur_analy).rename(columns = {"Price":"Total Purchase Value"})

#對 unique_sn 執行 df.groupby ， 得出 每種 Gender 的 Purchase count
Pur_analy["Purchase Count"] = data.groupby(['Gender']).Gender.count()

#將 Unique_player left join 到 Purchasing_analysis
Pur_analy = Pur_analy.join(Unique_player,how = 'left')

#計算 Average Purchase Price ，Avg Purchase Total per Person
Pur_analy["Average Purchase Price"] = (Pur_analy['Total Purchase Value'] / Pur_analy['Purchase Count']).round(5)
Pur_analy["Avg Purchase Total per Person"] = (Pur_analy['Total Purchase Value'] / Pur_analy['Player Count']).round(2)

#replace column values with function
#Avg Purchase Total per Person 加入錢字號
Pur_analy["Avg Purchase Total per Person"] = Pur_analy["Avg Purchase Total per Person"].map('${}'.format)

#重排 Purchasing_analysis column 順序
column_order = Pur_analy.columns.tolist()
column_order = column_order[1::2] + column_order[0::4]
Pur_analy = Pur_analy[column_order]
Pur_analy

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Gender,,,,
Female,113,3.20301,361.94,$4.47
Male,652,3.01785,1967.64,$4.07
Other / Non-Disclosed,15,3.34600,50.19,$4.56


In [6]:
#Age Demographics
#Categorize players using the age groups. Hint: use pd.cut()
#each group is for 4 years (i.e. <10, 10-14, 15-19, etc.)
#Calculate the numbers and percentages by age group
#Create a summary dataframe to hold the results
#round the percentage column to two decimal points
#Display Age Demographics Table

# 新增一個 "Age_cate" 的類別型 column  
player_age = unique_sn["Age"].values.tolist()
max_player_age = max(player_age)
min_player_age = min(player_age)
segment = [min_player_age - 1, 9, 14, 19, 24, 29, 34, 39, max_player_age]
label = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
unique_sn["Age_cate"] = pd.cut(unique_sn["Age"], bins=segment, labels = label)

#利用df.groupby 將 unique_sn 依照 Age_cate 做分類
age_demo = unique_sn.groupby("Age_cate",as_index=False).size()

#取出 column Age_cate 放入list，並將 column Age_cate 移除 
index_name = age_demo['Age_cate'].values.tolist()
age_demo = age_demo.drop('Age_cate', axis = 1)

# 製造 dict new_index，並對 age_demo 的 index 和 column重新命名
new_index = []
i = 0
while i < len(index_name):
    a = []
    a.append(i)
    a.append(index_name[i])
    new_index.append(a)
    i = i + 1

new_index = dict(new_index)
age_demo = age_demo.rename(index = new_index, columns = {"size":"Total Count"}) 

#插入 各類別數量的百分比
age_demo['Percentage of Players'] = (100 *  age_demo / age_demo.sum()).round(2) 

#調整 Gender demo column 順序
column_order = age_demo.columns.tolist()
column_order = column_order[-1:] + column_order[:-1]
age_demo = age_demo[column_order]
age_demo

<ipython-input-6-8878f9385dcb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_sn["Age_cate"] = pd.cut(unique_sn["Age"], bins=segment, labels = label)


,Percentage of Players,Total Count
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


In [7]:
#Purchasing Analysis (Age)
#The below each broken into groups of 4 years (i.e. <10, 10-14, 15-19, etc.)
#Purchase Count，Average Purchase Price，Total Purchase Value，Average Purchase Total per Person by Age Group
#Bin the purchase_data data frame by age
#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
#Create a summary data frame to hold the results
#Display the summary dataframe

# 新增一個 "Age_cate" 的類別型 column  
player_age = data["Age"].values.tolist()
max_player_age = max(player_age)
min_player_age = min(player_age)
segment = [min_player_age - 1, 9, 14, 19, 24, 29, 34, 39, max_player_age]
label = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
data["Age_cate"] = pd.cut(data["Age"], bins=segment, labels = label)

#利用df.groupby 將 data 依照 Age_cate 做個數加總
Pur_ana_age = data.groupby("Age_cate",as_index=False).size()

#對 data 執行 df.groupby ， 得出 每種 Age_cate 的 price 總和 
Total_Purchase_Value = data.groupby(['Age_cate'],as_index=False).Price.sum() 
Total_Purchase_Value = pd.DataFrame(Total_Purchase_Value)

#Pur_ana_age left join Total_Purchase_Value 的 column Price
Pur_ana_age = Pur_ana_age.join(Total_Purchase_Value["Price"], how = "left")

#取出 column Age_cate 放入list，並將 column Age_cate 移除 
index_name = Pur_ana_age['Age_cate'].values.tolist()
Pur_ana_age = Pur_ana_age.drop('Age_cate', axis = 1)

# 製造 dict new_index，並對 Pur_ana_age 的 index 和 column重新命名
new_index = []
i = 0
while i < len(index_name):
    a = []
    a.append(i)
    a.append(index_name[i])
    new_index.append(a)
    i = i + 1
new_index = dict(new_index)
Pur_ana_age = Pur_ana_age.rename(index = new_index, columns = {"size":"Purchase Count", "Price":"Total Purchase Value"}) 

#Pur_ana_age left join age_demo 的 column Total Count
Pur_ana_age = Pur_ana_age.join(age_demo["Total Count"],how = "left")

#計算 Average Purchase Price ，Avg Purchase Total per Person 
Pur_ana_age["Total Purchase Value"] = Pur_ana_age["Total Purchase Value"].round(2)
Pur_ana_age["Average Purchase Price"] = (Pur_ana_age['Total Purchase Value'] / Pur_ana_age['Purchase Count']).round(2)
Pur_ana_age["Avg Purchase Total per Person"] = (Pur_ana_age['Total Purchase Value'] / Pur_ana_age['Total Count']).round(2)

#replace column values with function
#add_price_mark 的 column 加入錢字號
add_price_mark = ["Total Purchase Value", "Average Purchase Price", "Avg Purchase Total per Person"] 
Pur_ana_age[add_price_mark] = Pur_ana_age[add_price_mark].applymap('${}'.format)

#重排 Purchasing_analysis column 順序
column_order = Pur_ana_age.columns.tolist()
column_order = column_order[0::3] + column_order[1::3]
Pur_ana_age = Pur_ana_age[column_order]
Pur_ana_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.9,$293.0,$3.81
30-34,73,$2.93,$214.0,$4.12
35-39,41,$3.6,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [8]:
#Top Spenders
#Identify the the top 5 spenders in the game by total purchase value 
#(top-5 players with highest total purchase value), then list (in a table):
#SN(screen name)，Purchase Count ， Average Purchase Price，Total Purchase Value
#Run basic calculations to obtain the results in the table below
#Create a summary dataframe to hold the results
#Sort the total purchase value column in descending order
#Display a preview of the summary dataframe

#利用df.groupby 將 data 依照 SN 做個數加總
top_sp = data.groupby("SN").size()
top_sp = pd.DataFrame(top_sp, columns = ["Purchase Count"])

#利用df.groupby 將 data 依照 SN 對Price加總
total_purchase_value = data.groupby("SN").Price.sum()
total_purchase_value = pd.DataFrame(total_purchase_value)

#top_sp left join total_purchase_value
top_sp = top_sp.join(total_purchase_value, how = "left")

#top_sp 依照 column Price 的值 ，做 降冪排序
top_sp = top_sp.sort_values(by = ["Price"], axis = 0, ascending = False)

#計算 Average Purchase Price
top_sp["Price"] = top_sp["Price"].round(2)
top_sp["Average Purchase Price"] = (top_sp["Price"]/top_sp["Purchase Count"]).round(2)

#top_sp column Price 重新命名成 Total Purchase Value
top_sp = top_sp.rename(columns = {"Price":"Total Purchase Value"})

#replace column values with function
#add_price_mark 的 column 加入錢字號
add_price_mark = ["Total Purchase Value", "Average Purchase Price"] 
top_sp[add_price_mark] = top_sp[add_price_mark].applymap('${}'.format)

#重排 top_sp column 順序 ，並顯示前五列
column_order = top_sp.columns.tolist()
column_order = column_order[0::2] + column_order[1::2]
top_sp = top_sp[column_order]
top_sp.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.4,$13.62
Iskadarya95,3,$4.37,$13.1


In [9]:
#Most Popular Items
#Top 5 most popular items by purchase count:
#Item ID ，Item Name ，Purchase Count , Item Price ，Total Purchase Value
#Retrieve the Item ID, Item Name, and Item Price columns
#Group by Item ID and Item Name. 
#Perform calculations to obtain purchase count, item price, and total purchase value
#Create a summary data frame to hold the results
#Sort the purchase count column in descending order
#Display a preview of the summary dataframe

# 清除 data column Item ID 的 重複值 
Item_price = data.drop_duplicates(subset = 'Item ID')

# 將 Item ID, Item Name 設成 multiindex
Item_price = Item_price.set_index(["Item ID", "Item Name"])

# 依照 index 重新排序
Item_price = Item_price.sort_index()

# 只留下 column Price 並 重新命名成 Item Price
Item_price = Item_price[["Price"]].rename(columns = {"Price": "Item Price"})

#利用df.groupby 將 data 依照 Item ID, Item Name 做個數加總
most_pop_items = data.groupby(["Item ID", "Item Name"]).size()
most_pop_items = pd.DataFrame(most_pop_items, columns = ['Purchase Count'])

#利用df.groupby 將 data 依照 Item ID, Item Name 對Price加總
#column Price 重新命名 Total Purchase Value
Total_Purchase_Value = data.groupby(["Item ID", "Item Name"]).Price.sum()
Total_Purchase_Value = pd.DataFrame(Total_Purchase_Value).rename(columns = {"Price": "Total Purchase Value"})

# most_pop_items left join Item_price ，Total_Purchase_Value
most_pop_items = most_pop_items.join([Item_price, Total_Purchase_Value], how = 'left')

# 將 column Total Purchase Value 四捨五入到小數第二位，並按照 column Purchase Count 降冪排序
most_pop_items["Total Purchase Value"] = most_pop_items["Total Purchase Value"].round(2)
most_pop_items = most_pop_items.sort_values(by = ["Purchase Count"], axis = 0, ascending = False)

#複製 most_pop_items
output = most_pop_items.copy()

#replace column values with function
#add_price_mark 的 column 加入錢字號
add_price_mark = ["Item Price", "Total Purchase Value"] 
output[add_price_mark] = output[add_price_mark].applymap('${}'.format)

output.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.9,$44.1
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [10]:
#Most Profitable Items
#Top 5 most profitable items by total purchase value:
#Item ID ，Item Name ，Purchase Count ，Item Price ，Total Purchase Value
#Sort the above table by total purchase value in descending order
#Display a preview of the dataframe

#按照 Total Purchase Value 降冪排序 
most_profit_items = most_pop_items.sort_values(by = ["Total Purchase Value"], axis = 0, ascending = False)

#replace column values with function
#add_price_mark 的 column 加入錢字號
add_price_mark = ["Item Price", "Total Purchase Value"] 
most_profit_items[add_price_mark] = most_profit_items[add_price_mark].applymap('${}'.format)

most_profit_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.9,$44.1
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.8
